In [3]:
import os,sys
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random

In [4]:
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
# 'WinHttpWriteData', #少了
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
api_li = [x.lower() for x in api_li] #lowrer case?小寫?
len(api_li)

26

In [6]:
corpus = []
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"\=\_]+\ *" #等號、底線被保留

root_dir = './data/tree-rep-profiles_o2o/normal/'
# rasmma_dir  = next(os.walk(root_dir))[1]
# for dir_ in rasmma_dir:
fam_dir = next(os.walk(root_dir ))[1]
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir + fam))[1]
    for tree in tree_dir:
        in_directory = root_dir  + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        for profile in hl_list:
            with open(profile,encoding='ISO 8859-1') as f:
                lines = f.read()
            lines = re.sub(r'[^\x00-\x7F]+','', lines)
            lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
            lines = lines.splitlines()
            for line in lines:
                temp = re.sub(dil," ",line.lower())#.lower()) #小寫?
                temp = temp.split(" ")
                temp = list(filter(None, temp))
                temp = ' '.join(temp)
                if temp.startswith('winh'.lower()):
                    print(profile,temp)
                if temp.split(' ')[0] not in api_li:
                    print('=o2o_ERR:=',profile,'=>',temp)
                    continue
                corpus.append(temp)

with open('data/sent2vec_corpus/o2o_Sent2Vec_lower_woParam_0630.txt','w') as f:
    f.write('\n'.join(corpus))
corpus[-10:]

100%|██████████| 130/130 [00:06<00:00, 20.83it/s]


['createfile pr sys exe pr generic read pr open existing pr file share delete file share read ret 0',
 'loadlibrary pr sys explorer exe ret 0',
 'createfile pr sys exe pr generic read pr open existing pr file share delete file share read ret 0',
 'regqueryvalue pr hkcu soft ms win explorer shellfolders pr subk cache pr 0 pr 12e8f4 ret 0',
 'loadlibrary pr sys advapi32 dll ret 0',
 'loadlibrary pr sys oleaut32 dll ret 0',
 'regqueryvalue pr hklm software microsoft windows html help pr subk hlp pr 0 pr cb0398 ret p',
 'regqueryvalue pr hklm software microsoft windows help pr subk hlp pr 0 pr cb0398 ret p',
 'exitprocess pr 0',
 'deletefile pr usr tmp ret 0']

# Sent2Vec Training time 
* cd ~/Downloads/sent2vec/

In [ ]:
with open('data/sent2vec_corpus/o2o_Sent2Vec_lower_woParam_0630.txt','r') as f: #改
    corpus = f.read().splitlines()
corpus = sorted(corpus)
corpus[-100:]

In [ ]:
model = sent2vec.Sent2vecModel()
model.load_model('model/o2o_o2m_Sent2Vec_woParam_0616.bin') #改
emb_all = model.embed_sentences(corpus)

In [ ]:
api = ""
count = 0
count_all = {}
for row in corpus:
    try:
        count_all[row.split(' ')[0]] += 1
    except KeyError:
        count_all[row.split(' ')[0]] = 1
print(len(count_all))
sorted(count_all.items())

In [ ]:
kk = list(count_all.keys())
for k in api_li:
    if k not in kk:
        print('少了:',k)
for k in kk:
    if k not in api_li:
        print('多了',k)

In [ ]:
c= 0
df = pd.DataFrame(0, index=api_li, columns=api_li)
index_ = df.index.tolist()
avg = 100
for _ in range(avg):
    c=0
    for k,v in count_all.items():
        pick = random.randrange(c,c+v) #cosine similarity的左邊
        pick = emb_all[pick,:] #sorted corpus的embedding_all
        c1 = 0
        try:
            row = index_.index(k)
        except ValueError:
            continue
        for k1,v1 in count_all.items(): # cosine similarity的右邊
            pick1 = random.randrange(c1,c1+v1)
            pick1 = emb_all[pick1,:]
            score = cosine_similarity([pick],[pick1]) #cosine_similarity
    #         print(score)
            c1 = c1+v1
            try:
                col = index_.index(k1)
                df.iloc[row,col] += float(score)
            except ValueError:
                continue
        c = c+v
df = df/avg #similarity
# df = (df-np.min(df.values))/(np.max(df.values)-np.min(df.values)) #distance normalize
df

In [ ]:
lib = ['LoadLibrary']
proc = ['CreateProcess', 'OpenProcess', 'ExitProcess', 'TerminateProcess', 'WinExec',
        'CreateRemoteThread', 'CreateThread']
file = ['CopyFile', 'CreateFile', 'DeleteFile']
reg = ['RegSetValue', 'RegCreateKey', 'RegDeleteKey', 'RegDeleteValue',
       'RegQueryValue', 'RegEnumValue']
net = ['WinHttpConnect', 'WinHttpOpen', 'WinHttpOpenRequest', 'WinHttpReadData', 'WinHttpSendRequest', #'WinHttpWriteData',
        'InternetOpen', 'InternetConnect', 'HttpSendRequest', 'GetUrlCacheEntryInfo']

#lower?
lib = [x.lower() for x in lib]
proc = [x.lower() for x in proc]
file = [x.lower() for x in file]
reg =[x.lower() for x in reg]
net = [x.lower() for x in net]

# df.loc['']
x = [lib,proc,file,reg,net]
x = sum(x,[])
index_ , len(index_)

In [ ]:
up=0
down = len(index_)
down_cat = len(x)
up_cat = 0
up_cat2 = 0
lib_lib=lib_proc=lib_file=lib_reg=lib_net = 0
proc_lib=proc_proc=proc_file=proc_reg=proc_net = 0
file_lib=file_proc=file_file=file_reg=file_net = 0
reg_lib=reg_proc=reg_file=reg_reg=reg_net = 0
net_lib=net_proc=net_file=net_reg=net_net = 0
for api in index_:
    if df.loc[api].idxmax() == api:
        up+=1
    else:
        print(api)
    baseline_lib = df.loc[api,lib].mean()
    baseline_proc =  df.loc[api,proc].mean()
    baseline_file = df.loc[api,file].mean()
    baseline_reg = df.loc[api,reg].mean()
    baseline_net = df.loc[api,net].mean()
    max_value = np.max([baseline_lib,baseline_proc,baseline_file,baseline_reg,baseline_net])
    if (api in lib):
        lib_lib += baseline_lib
        lib_proc += baseline_proc
        lib_file += baseline_file
        lib_reg += baseline_reg
        lib_net += baseline_net
        if (max_value==baseline_lib):
            up_cat+=1
    elif api in proc :
        proc_lib += baseline_lib
        proc_proc += baseline_proc
        proc_file += baseline_file
        proc_reg += baseline_reg
        proc_net += baseline_net
        if max_value==baseline_proc:
            up_cat+=1
    elif api in file :
        file_lib += baseline_lib
        file_proc += baseline_proc
        file_file += baseline_file
        file_reg += baseline_reg
        file_net += baseline_net
        if max_value==baseline_file:
            up_cat+=1    
    elif api in reg:
        reg_lib += baseline_lib
        reg_proc += baseline_proc
        reg_file += baseline_file
        reg_reg += baseline_reg
        reg_net += baseline_net
        if max_value==baseline_reg:
            up_cat+=1    
    elif api in net:
        net_lib += baseline_lib
        net_proc += baseline_proc
        net_file += baseline_file
        net_reg += baseline_reg
        net_net += baseline_net
        if max_value==baseline_net:
            up_cat+=1
if np.max([lib_lib,lib_proc,lib_file,lib_reg,lib_net]) == lib_lib:
    up_cat2+=1
if np.max([proc_lib,proc_proc,proc_file,proc_reg,proc_net]) == proc_proc:
    up_cat2+=1
if np.max([file_lib,file_proc,file_file,file_reg,file_net]) == file_file:
    up_cat2+=1
if np.max([reg_lib,reg_proc,reg_file,reg_reg,reg_net]) == reg_reg:
    up_cat2+=1
if np.max([net_lib,net_proc,net_file,net_reg,net_net]) == net_net:
    up_cat2+=1
        
print('自己還原自己-ACC:',(up/down)*100,'%') #self
print('自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的):',(up_cat/down_cat)*100,'%') #micro
print('自己那類屬於自己那一類-ACC:',(up_cat2/5)*100,'%') #macro

In [ ]:
score = []
for i in range(len(df)):
    score.append(df.iloc[i,i])
print('對角線分數(越高越好):',np.mean(score))